In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer  # Import Tokenizer
import time

In [2]:
# Load the IMDB dataset
max_features = 5000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)


In [3]:
# Preprocess the data
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [4]:
# Convert sequences back into text
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)

# Preprocess the data
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=maxlen)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=maxlen)

In [5]:
# Create and fit tokenizer
#tokenizer = Tokenizer(num_words=max_features)
#tokenizer.fit_on_texts(X_train)


In [6]:
# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
# Train the model
batch_size = 32
epochs = 5
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

In [9]:
start_time = time.time()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)
training_time = time.time() - start_time


Epoch 1/5
782/782 [==============================] - 166s 204ms/step - loss: 0.4178 - accuracy: 0.8062 - val_loss: 0.3453 - val_accuracy: 0.8471
Epoch 2/5
782/782 [==============================] - 165s 212ms/step - loss: 0.2968 - accuracy: 0.8765 - val_loss: 0.3574 - val_accuracy: 0.8438
Epoch 3/5
782/782 [==============================] - 169s 216ms/step - loss: 0.2489 - accuracy: 0.8991 - val_loss: 0.3728 - val_accuracy: 0.8408
Epoch 4/5
782/782 [==============================] - 156s 200ms/step - loss: 0.2115 - accuracy: 0.9176 - val_loss: 0.3867 - val_accuracy: 0.8455
Epoch 5/5
782/782 [==============================] - 123s 157ms/step - loss: 0.1771 - accuracy: 0.9306 - val_loss: 0.4303 - val_accuracy: 0.8446


In [10]:
# Evaluate the model
start_time = time.time()

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
prediction_time = time.time() - start_time

print("Test accuracy:", acc)

782/782 [==============================] - 15s 19ms/step - loss: 0.4303 - accuracy: 0.8446
Test accuracy: 0.8446400165557861


In [11]:
# Function to preprocess user input
def preprocess_input(user_input):
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)
   


In [12]:
# Function to predict sentiment
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "Positive"
    else:
        return "Negative"

In [13]:
# User Interface
print("Welcome to Movie Review Sentiment Analyzer!")

Welcome to Movie Review Sentiment Analyzer!


In [ ]:
while True:
    user_review = input("Enter your movie review (or type 'quit' to exit): ")
    if user_review.lower() == 'quit':
        print("Thank you for using our service!")
        break
    else:
        sentiment = predict_sentiment(user_review)
        print("Predicted sentiment:", sentiment)

Enter your movie review (or type 'quit' to exit): better
1/1 [==============================] - 0s 285ms/step
Predicted sentiment: Negative
Enter your movie review (or type 'quit' to exit): best
1/1 [==============================] - 0s 17ms/step
Predicted sentiment: Positive
Enter your movie review (or type 'quit' to exit): good
1/1 [==============================] - 0s 31ms/step
Predicted sentiment: Negative
Enter your movie review (or type 'quit' to exit): bad
1/1 [==============================] - 0s 29ms/step
Predicted sentiment: Negative
Enter your movie review (or type 'quit' to exit): very bad
1/1 [==============================] - 0s 18ms/step
Predicted sentiment: Negative
Enter your movie review (or type 'quit' to exit): excellent
1/1 [==============================] - 0s 16ms/step
Predicted sentiment: Positive
Enter your movie review (or type 'quit' to exit): amazing
1/1 [==============================] - 0s 26ms/step
Predicted sentiment: Positive
Enter your movie review (or

In [16]:
print("Test accuracy:", acc)
print("Training time:", training_time, "seconds")
print("Prediction time:", prediction_time, "seconds")


Test accuracy: 0.8469600081443787
Training time: 758.8439946174622 seconds
Prediction time: 19.827228784561157 seconds
